In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]

In [3]:
df = pd.read_excel(PROJECT_ROOT / "data/raw/ventes MT par secteurs d'activité.xlsx")

In [4]:
df.head()

,Filtre darrière-plan:,Type d'usage {MTGEN : MTGEN; MTV : MTV; MTGENBO : MTGENBO},Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,Unnamed: 288,Unnamed: 289
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Ratios,Année civile,Année civile/mois,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,...,KWh,KWh,KWh,KWh,KWh,KWh,KWh,KWh,KWh,KWh
3,NaN,NaN,2007,2007,2007,2007,2007,2007,2007,2007,...,2017,2017,2017,2017,2017,2017,2017,2017,2017,Résultat global
4,DR_DP_AS,Activités,01.2007,02.2007,03.2007,04.2007,05.2007,06.2007,07.2007,08.2007,...,05.2017,06.2017,07.2017,08.2017,09.2017,10.2017,11.2017,12.2017,Résultat,Résultat


In [5]:
import sqlite3
db_orig = sqlite3.connect(PROJECT_ROOT / "data/ONEE_Regional_COMPLETE.db")
df_regional_orig = pd.read_sql_query("SELECT * FROM monthly_data", db_orig)
db_orig.close()

In [6]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [9]:
def extract_city(s):
    return s.split(':')[-1].strip().split()[-1]
unique = set()
for s in df.iloc[5:]["Filtre darrière-plan:"]:
    city = extract_city(s)
    if city not in unique:
        unique.add(city)

In [10]:
unique

{'AGADIR',
 'CASA',
 'ERRACHIDIA',
 'FES',
 'LAAYOUNE',
 'MARRAKECH',
 'MELLAL',
 'OUJDA',
 'RABAT',
 'TANGER',
 'global',
 'régional(n/e)'}

In [11]:
df_regional_orig["City"].unique()

array(['agadir', 'beni_mellal', 'casablanca', 'errachidia', 'fes',
       'laayoune', 'marrakech', 'oujda', 'rabat', 'tanger'], dtype=object)

In [13]:
df = df.drop(index= [0,1,3]).reset_index(drop=True)

In [16]:
df.columns = df.iloc[0].fillna('') + '__' + df.iloc[1].fillna('')

In [18]:
df.columns = df.columns.str.strip('_')
df.columns = df.columns.str.strip()
df = df.drop([0, 1])

In [23]:
df = df.drop([52, 53])

In [28]:
activity_mapping = {
    '01- Agricole': 'Agricole',
    '02- Industriel': 'Industriel',
    '03- Tertaire': 'Tertiaire',
    '05- Administratif': 'Administratif',
    '06- Résidentiel': 'Résidentiel'
}

In [29]:
df["Activités"] = df["Activités"].replace(activity_mapping)

In [34]:
city_mapping = {
    'AGADIR': 'agadir',
    'CASA': 'casablanca',
    'ERRACHIDIA': 'errachidia',
    'FES': 'fes',
    'LAAYOUNE': 'laayoune',
    'MARRAKECH': 'marrakech',
    'MELLAL': 'beni_mellal',
    'OUJDA': 'oujda',
    'RABAT': 'rabat',
    'TANGER': 'tanger'
}

In [ ]:
df["DR_DP_AS"] = df["DR_DP_AS"].apply(lambda s: city_mapping[extract_city(s)])

In [40]:
df = df.groupby(["DR_DP_AS","Activités"], as_index = False).sum()

C:\Users\Soufiane.AITELAOUAD\AppData\Local\Temp\ipykernel_9828\453937611.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = df.groupby(["DR_DP_AS","Activités"], as_index = False).sum()
C:\Users\Soufiane.AITELAOUAD\AppData\Local\Temp\ipykernel_9828\453937611.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = df.groupby(["DR_DP_AS","Activités"], as_index = False).sum()


In [42]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [43]:
import re
import pandas as pd

id_cols = ['DR_DP_AS', 'Activités']

val_cols = [c for c in df.columns if re.match(r'^.+__\d{2}\.\d{4}$', str(c))]

long = df.melt(id_vars=id_cols, value_vars=val_cols,
               var_name='key', value_name='value')

parts = long['key'].str.extract(r'^(?P<metric>.+?)__(?P<Month>\d{2})\.(?P<Year>\d{4})')
long = pd.concat([long.drop(columns='key'), parts], axis=1)
long['Month'] = long['Month'].astype(int)
long['Year'] = long['Year'].astype(int)
long['metric'] = long['metric'].str.strip()

wide = (long.pivot_table(index=id_cols+['Year','Month'],
                         columns='metric', values='value', aggfunc='first')
             .reset_index())

wide = wide.rename(columns={'NB_CLIENTS':'Nbr Clients', 'KWh':'KWh'})

In [44]:
wide

metric,DR_DP_AS,Activités,Year,Month,KWh,Nbr Clients
0,agadir,Administratif,2007,1,1196854,58
1,agadir,Administratif,2007,2,1063830,58
2,agadir,Administratif,2007,3,1147537,55
3,agadir,Administratif,2007,4,1072214,55
4,agadir,Administratif,2007,5,1122329,57
...,...,...,...,...,...,...
6595,tanger,Tertiaire,2017,8,2405821,309
6596,tanger,Tertiaire,2017,9,2319671,309
6597,tanger,Tertiaire,2017,10,1854111,312
6598,tanger,Tertiaire,2017,11,1708445,312


In [47]:
6600/(12 * 11)

50.0

In [48]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [49]:
wide.columns

Index(['DR_DP_AS', 'Activités', 'Year', 'Month', 'KWh', 'Nbr Clients'], dtype='object', name='metric')

In [50]:
wide["KWh"] = wide["KWh"] / 1000

In [51]:
wide.rename(columns = {"Activités":"Activity", "DR_DP_AS":"City", "KWh": "MWh"}, inplace = True)

In [53]:
city_to_region = {
    'beni_mellal': 'Béni Mellal-Khénifra',
    'casablanca': 'Casablanca-Settat',
    'errachidia': 'Drâa-Tafilalet',
    'fes': 'Fès-Meknès',
    'laayoune': 'Laâyoune-Sakia El Hamra',
    'marrakech': 'Marrakech-Safi',
    'oujda': 'Oriental',
    'rabat': 'Rabat-Salé-Kénitra',
    'agadir': 'Souss-Massa',
    'tanger': 'Tanger-Tétouan-Al Hoceïma'
}


In [54]:
wide["Region"] = wide["City"].apply(lambda x: city_to_region[x])

In [80]:
wide_2017 = wide.copy()

## 2018 - 2023

In [58]:
dff = pd.read_excel(PROJECT_ROOT / "data/raw/Ventes en MWh  MT.xlsx")

In [63]:
dff = dff.drop(index =[50])

In [65]:
dff["Activités"] = dff["Activités"].replace(activity_mapping)

In [67]:
dff.head()

,Activités,DR_DP_AS,01.2018,02.2018,03.2018,04.2018,05.2018,06.2018,07.2018,08.2018,...,03.2023,04.2023,05.2023,06.2023,07.2023,08.2023,09.2023,10.2023,11.2023,12.2023
0,Agricole,Direction Rég. CASA,8007.470667,7836.817667,8374.818667,13064.997333,13631.351333,13207.392333,12811.567000,12100.497000,...,10639.022333,10339.122667,11828.122667,11793.578667,12209.395000,13308.400000,10740.034000,10941.977667,9889.562667,10288.050667
1,Agricole,Direction Rég. RABAT,12202.349667,11911.737667,14471.513667,16886.515667,23183.826667,27736.169667,32218.183000,31562.393000,...,15115.246333,26364.999000,23195.336000,25612.983000,31202.521000,30578.100000,24189.850000,19370.960333,14316.909333,13481.618333
2,Agricole,Direction Rég. ERRACHIDIA,676.223667,700.094667,1063.530667,1292.269000,1336.961000,1845.875000,2251.164667,1721.063667,...,3127.476000,3857.663333,4636.137333,5110.481333,6357.059000,6747.045000,5164.218000,4380.570000,3219.228000,2897.028000
3,Agricole,Direction Rég. FES,6742.345000,5545.538000,5432.540000,6084.858000,8083.173000,12909.353000,18934.778333,20700.304333,...,9075.153000,14667.473000,14842.817000,17173.468000,23683.297333,23370.003333,16551.626333,13713.109667,10841.046667,10378.890667
4,Agricole,Direction Rég. OUJDA,1766.963000,1428.980000,2499.345000,3143.420333,4221.614333,5349.589333,6710.052667,6440.106667,...,5816.840333,6683.051667,7194.444667,7207.576667,8827.747333,9187.794333,5376.488333,5979.644333,5529.627333,5459.122333


In [69]:
dff["DR_DP_AS"] = dff["DR_DP_AS"].apply(lambda s: city_mapping[extract_city(s)])

In [76]:
dff = dff.groupby(["DR_DP_AS","Activités"], as_index = False).sum()

In [84]:
cols = list(dff.columns)

In [85]:
for i in range(2, len(cols)):
    cols[i] = "KWh__" + cols[i]

In [86]:
dff.columns = cols

In [88]:
import re
import pandas as pd

id_cols = ['DR_DP_AS', 'Activités']

val_cols = [c for c in dff.columns if re.match(r'^.+__\d{2}\.\d{4}$', str(c))]

long = dff.melt(id_vars=id_cols, value_vars=val_cols,
               var_name='key', value_name='value')

parts = long['key'].str.extract(r'^(?P<metric>.+?)__(?P<Month>\d{2})\.(?P<Year>\d{4})')
long = pd.concat([long.drop(columns='key'), parts], axis=1)
long['Month'] = long['Month'].astype(int)
long['Year'] = long['Year'].astype(int)
long['metric'] = long['metric'].str.strip()

wide = (long.pivot_table(index=id_cols+['Year','Month'],
                         columns='metric', values='value', aggfunc='first')
             .reset_index())

wide = wide.rename(columns={'KWh':'KWh'})

In [ ]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [92]:
wide.columns

Index(['DR_DP_AS', 'Activités', 'Year', 'Month', 'KWh'], dtype='object', name='metric')

In [93]:
wide.rename(columns = {"Activités":"Activity", "DR_DP_AS":"City", "KWh": "MWh"}, inplace = True)

In [95]:
wide["Region"] = wide["City"].apply(lambda x: city_to_region[x])

## Combining

In [99]:
final_df = pd.concat([wide_2017, wide]).sort_values(by = ["Year", "Month"])

In [101]:
final_df.to_csv(PROJECT_ROOT / "data/srm_data_2007_2023_mt.csv", index = False)

In [103]:
import sqlite3

# Path to your existing SQLite database
db_path = PROJECT_ROOT / "data/ONEE_Regional_COMPLETE_2007_2023.db"

# Connect to the existing SQLite database
conn = sqlite3.connect(db_path)

# Name of the new table you want to create
table_name = 'monthly_data_mt'

# Write the DataFrame to the new table
final_df.to_sql(table_name, conn, if_exists='replace', index=False)

conn.close()
